# YouTube Ranking

In [59]:
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from tqdm.notebook import tqdm

# 1. 1페이지만 정보 추출

In [60]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube'
driver = webdriver.Chrome('chromedriver.exe')
driver.get(url)
time.sleep(2)

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_7516\3572365892.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')


* step1, 2

In [4]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
trs = soup.select('tr.aos-init')
len(trs)

100

* step3

In [5]:
# 랭킹
tr = trs[0]
tr.select_one('.rank').get_text().strip()

'1'

In [14]:
# 카테고리
tr.select_one('.category').get_text().strip()[1:-1]

'게임'

In [7]:
# 채널명
tr.select_one('.subject a').get_text().strip()

'BLACKPINK'

In [11]:
# 구독자, 조회수, 영상수
subscriber = tr.select_one('.subscriber_cnt').get_text().strip()
view = tr.select_one('.view_cnt').get_text().strip()
video = tr.select_one('.video_cnt').get_text().strip()
subscriber, view, video

('8380만', '286억1398만', '466개')

In [15]:
lines = []
for tr in trs:
    rank = tr.select_one('.rank').get_text().strip()
    category = tr.select_one('.category').get_text().strip()[1:-1]
    chennel = tr.select_one('.subject a').get_text().strip()
    subscriber = tr.select_one('.subscriber_cnt').get_text().strip()
    view = tr.select_one('.view_cnt').get_text().strip()
    video = tr.select_one('.video_cnt').get_text().strip()
    lines.append([rank, category, chennel, subscriber, view, video])

In [16]:
df = pd.DataFrame(lines, columns=['순위', '카테고리', '채널명', '구독자수', '조회수', '영상수'])
df

,순위,카테고리,채널명,구독자수,조회수,영상수
0,1,음악/댄스/가수,BLACKPINK,8380만,286억1398만,466개
1,2,음악/댄스/가수,BANGTANTV,7300만,191억4327만,"2,086개"
2,3,음악/댄스/가수,HYBE LABELS,6950만,258억4674만,"1,065개"
3,4,음악/댄스/가수,SMTOWN,3130만,262억1822만,"4,052개"
4,5,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],2650만,110억5288만,223개
...,...,...,...,...,...,...
95,96,TV/방송,JTBC Drama,469만,50억8639만,"27,000개"
96,98,키즈/어린이,CoCosToy 꼬꼬스토이,467만,28억7649만,656개
97,97,음식/요리/레시피,Cooking tree 쿠킹트리,467만,4억6943만,"1,410개"
98,99,음식/요리/레시피,까니짱 [ G-NI : 蟹ちゃん],465만,13억5018만,719개


In [41]:
def convert_unit(string):
    s = string.replace('억', '').replace('만', '0000').replace('개', '').replace(',', '')
    return f'{int(s):,d}'

In [42]:
convert_unit('286억1398만')

'28,613,980,000'

In [43]:
for column in df.columns[3:]:
    df[column] = df[column].apply(convert_unit)

In [44]:
df

,순위,카테고리,채널명,구독자수,조회수,영상수
0,1,음악/댄스/가수,BLACKPINK,"83,800,000","28,613,980,000",466
1,2,음악/댄스/가수,BANGTANTV,"73,000,000","19,143,270,000","2,086"
2,3,음악/댄스/가수,HYBE LABELS,"69,500,000","25,846,740,000","1,065"
3,4,음악/댄스/가수,SMTOWN,"31,300,000","26,218,220,000","4,052"
4,5,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],"26,500,000","11,052,880,000",223
...,...,...,...,...,...,...
95,96,TV/방송,JTBC Drama,"4,690,000","5,086,390,000","27,000"
96,98,키즈/어린이,CoCosToy 꼬꼬스토이,"4,670,000","2,876,490,000",656
97,97,음식/요리/레시피,Cooking tree 쿠킹트리,"4,670,000","469,430,000","1,410"
98,99,음식/요리/레시피,까니짱 [ G-NI : 蟹ちゃん],"4,650,000","1,350,180,000",719


# 2. 10페이지까지 정보추출

In [61]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube'
lines = []
options = webdriver.ChromeOptions()
options.add_argument("headless")
for i in tqdm(range(0,10)):
    if i == 0:
        url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube'
    else:
        url = soup.select('.pg a')[i-1]['href']
    driver = webdriver.Chrome('chromedriver.exe', options=options)
    driver.get(url)
    time.sleep(5)
    BeautifulSoup(driver.page_source, 'html.parser')
    soup.select('tr.aos-init')
    
    for tr in trs:
        rank = tr.select_one('.rank').get_text().strip()
        category = tr.select_one('.category').get_text().strip()[1:-1]
        chennel = tr.select_one('.subject a').get_text().strip()
        subscriber = tr.select_one('.subscriber_cnt').get_text().strip()
        view = tr.select_one('.view_cnt').get_text().strip()
        video = tr.select_one('.video_cnt').get_text().strip()
        lines.append([rank, category, chennel, subscriber, view, video])

  0%|          | 0/10 [00:00<?, ?it/s]

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_7516\1039895744.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe', options=options)


In [62]:
df = pd.DataFrame(lines, columns=['순위', '카테고리', '채널명', '구독자수', '조회수', '영상수'])
df

,순위,카테고리,채널명,구독자수,조회수,영상수
0,1,음악/댄스/가수,BLACKPINK,8380만,286억1398만,466개
1,2,음악/댄스/가수,BANGTANTV,7300만,191억4327만,"2,086개"
2,3,음악/댄스/가수,HYBE LABELS,6950만,258억4674만,"1,065개"
3,4,음악/댄스/가수,SMTOWN,3130만,262억1822만,"4,052개"
4,5,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],2650만,110억5288만,223개
...,...,...,...,...,...,...
995,96,TV/방송,JTBC Drama,469만,50억8639만,"27,000개"
996,98,키즈/어린이,CoCosToy 꼬꼬스토이,467만,28억7649만,656개
997,97,음식/요리/레시피,Cooking tree 쿠킹트리,467만,4억6943만,"1,410개"
998,99,음식/요리/레시피,까니짱 [ G-NI : 蟹ちゃん],465만,13억5018만,719개
